In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# CFU reduction C873 Replacement
c873_update = pd.read_csv(#redacted.csv)
cfu_reduction_raw = pd.read_csv(#redacted.csv)
finalCFUset = pd.read_csv(#redacted.csv)
LB_CFUset = pd.read_csv(#redacted.csv)
LB_strains = list(LB_CFUset.Strain.unique())
cfu_reduction_raw = cfu_reduction_raw[cfu_reduction_raw["Strain"] != "C000873"]
cfu_reduction_raw = cfu_reduction_raw[~cfu_reduction_raw.Strain.isin(LB_strains)]
cfu_reduction = pd.concat([cfu_reduction_raw, c873_update, finalCFUset, LB_CFUset], axis=0)

cfu_reduction.drop("Standard_Deviation", axis=1, inplace=True)
cfu_reduction.drop("EXP", axis=1, inplace=True)
cfu_reduction.Phage = cfu_reduction.Phage.apply(lambda x: "Bactrim" if (x == "SXT") | (x == "Media-Bactrim") else x)
cfu_reduction.Phage = cfu_reduction.Phage.apply(lambda x: "No Phage" if (x == "Media") | (x == "Blank")  else x)
cfu_reduction.Phage = cfu_reduction.Phage.apply(lambda x: "CK618" if (x == "LBP-EC01") | (x == "CK000618") else x)
cfu_reduction.Phage = cfu_reduction.Phage.apply(lambda x: "CK618_bactrim" if (x == 'CK000618-Bactrim') | (x == 'LBP-EC01 + Bactrim') | (x == 'CK618-SXT') else x)
cfu_reduction_avgs = cfu_reduction.groupby(["Phage", "Strain", "Timepoint"])["count_log_titer"].agg(['mean', np.std,]).reset_index()
cfu_reduction_avgs.columns = ["Phage", "Strain", "Timepoint", "log_titer", "StDev"]

#drop rows that aren't a c number strain
cfu_reduction_avgs = cfu_reduction_avgs[cfu_reduction_avgs.Strain.str.startswith("C")]

print(cfu_reduction_avgs.Phage.unique())
print(cfu_reduction_avgs.Strain.unique())
print("Strains tested: " + str(len(cfu_reduction_avgs.Strain.unique())))

#check for duplicates
duplicates = cfu_reduction_avgs.duplicated(subset=['Phage', 'Strain', 'Timepoint'], keep=False)
#print(cfu_reduction_avgs[duplicates].Strain.unique())
#no duplicates, good.

#specify media for the strains
cfu_reduction_avgs['media'] = cfu_reduction_avgs.Strain.apply(lambda x: 'LB' if x in LB_strains else 'urine')

cfu_reduction_avgs.to_csv(#redacted.csv, index=False)

In [ ]:
print(finalCFUset.Strain.unique())
original_strains = list(cfu_reduction_raw.Strain.unique())
new_strains = finalCFUset.Strain.unique()
repeat_strains = [strain for strain in new_strains if strain in original_strains]
print(repeat_strains)

In [ ]:
print(LB_CFUset.Strain.unique())
original_strains = list(cfu_reduction_raw.Strain.unique())
new_strains = LB_CFUset.Strain.unique()
repeat_strains = [strain for strain in new_strains if strain in original_strains]
print(repeat_strains)

In [ ]:
#new dataset avergaes and stuff
old_stuff_avgs = finalCFUset.groupby(["Phage", "Strain", "Timepoint"])["count_log_titer"].agg(['mean', np.std,]).reset_index()
old_stuff_avgs.columns = ["Phage", "Strain", "Timepoint", "log_titer", "StDev"]
# cfu_reduction_avgs.Phage = cfu_reduction_avgs.Phage.apply(lambda x: "CK618" if (x == "LBP-EC01") | (x == "CK000618") else x)
# cfu_reduction_avgs.Phage = cfu_reduction_avgs.Phage.apply(lambda x: "CK618_bactrim" if (x == 'CK000618-Bactrim') | (x == 'LBP-EC01 + Bactrim') else x)
repeat_strains_subset = cfu_reduction_avgs[(old_stuff_avgs["Strain"] == "C000873") | (cfu_reduction_avgs["Strain"] == "C000835")]
new_repeat_subset =  cfu_reduction_avgs[(cfu_reduction_avgs["Strain"] == "C000873") | (cfu_reduction_avgs["Strain"] == "C000835")]

# Old and new data look consistent so averaging it all seems valid